<a href="https://colab.research.google.com/github/columbia-data-club/meetings/blob/main/2023/march_09_python_and_relational_databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A blue background with the SQLite logo and the words Data Club on it](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/data-club-sqlite.png)

# Python & Relational Databases

March 9, 2023

by [Moacir P. de Sá Pereira](https://moacir.com) for the [Columbia Data Club](https://github.com/columbia-data-club/), using data from the [Pokemon-Database](https://github.com/brianr852/Pokemon-Database) on GitHub, forked to [@columbia-data-club/Pokemon-Database](https://github.com/columbia-data-club/Pokemon-Database).

This notebook provides an introduction to relational databases in general and to using them with pandas in particular. A basic understanding of Python syntax (such as the one covered in the Data Club’s [Intro to Python video](https://youtu.be/l45rzo4MUHs) should suffice. 

## What Is a Relational Database?

Simply, but perhaps unhelpfully, a database is a collection of data where the data are structured and grouped into relations. I think it might be easiest to show this relationship using a pretend taxi dataset based on the one we used last week.

First, let’s install the [Faker](https://faker.readthedocs.io/en/master/) library, which will let us generate fake data. While we’re here, let’s also install [Pony](https://ponyorm.org), which we’ll be using in the second half of the workshop.

In [ ]:
!python -m pip install Faker
!python -m pip install pony

Now let’s create a simple dataframe that looks like the one we have been using. We’ll create a pickup time, a dropoff time, a fare, a distance, a number of passengers, and a tip amount.

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
from faker import Faker
fake = Faker()

# Set the random seeds so we get the same results between us
Faker.seed(42)
np.random.seed(42)

# Let's define the NYC fare calculator once:
#
# NYC taxis cost $1.56 per km, have a $2.50 minimum, 
# and also charge $.50 per minute of waiting.
# let's assume an ideal, wait-free speed of 35kph
def fare_calculator(distance, duration): # distance in km, duration in min
  ideal_duration = distance/(35/60)
  wait_charge = 0.5 * (duration - ideal_duration)
  if(wait_charge < 0): # wait charge cannot be negative
    wait_charge = 0
  total_charge = wait_charge + distance * 1.56
  if(total_charge > 2.5): # total cannot be below $2.50
    return round(total_charge, 2)
  return 2.5

def build_taxi_data():
  pickup_dt = fake.date_time_between(start_date='-15d', end_date='now')
  # let's assume trips are 20 minutes long on average, but usually shorter
  # (the +1 guarantees no 0 minute trips)
  duration_in_minutes = np.random.poisson(19) + 1
  dropoff_dt = pickup_dt + dt.timedelta(minutes=duration_in_minutes)
  # let's assume taxis travel about 20kph, 
  # with taxis rarely having an average speed below 10kph
  distance_km = (duration_in_minutes / 60) * np.random.normal(loc=20, scale=5)
  fare = fare_calculator(distance_km, duration_in_minutes)
  # Assume 1.4 passengers on average, but correct for 0s:
  random_passenger_count = np.random.poisson(1.4)
  passengers = random_passenger_count if random_passenger_count > 0 else 1
  return {
      'pickup_dt': pickup_dt,
      'dropoff_dt': pickup_dt + dt.timedelta(minutes=duration_in_minutes),
      'distance_km': distance_km,
      # Assume about 1 passenger on average, but no 0s.
      'passengers': passengers,
      'fare': fare,
      # Assume an average tip of 15%
      'tip': round(np.random.normal(loc=0.15, scale=0.05) * fare, 2)
  }

data = [build_taxi_data() for _ in range(100)]
df = pd.DataFrame(data)
# Let's also add the average speed of the taxi per ride.
df["speed_kph"] = df.apply(lambda x: x["distance_km"] / ((x.dropoff_dt - x.pickup_dt).seconds / 3600), axis=1)
df.head()


### Time to add relationality?

So far so good, and the dataframe looks a lot like the one we used two weeks ago. But let’s assume this isn’t New York City taxi data, but, rather, the taxi data specifically for Master Splinter’s Taxi Company. Master Splinter has four drivers, and he wants to keep track of the rides each driver gives. Let’s randomly assign each ride to one of his drivers.

Can you think of a reason why assigning drivers randomly would not work in the real world?

In [ ]:
# We need to declare the seed in every cell.
# Here I'm using a slightly different one for narrative purposes.
np.random.seed(46)
drivers = ["Leonardo", "Michelangelo", "Raphael", "Donatello"]
df['driver_id'] = [np.random.choice(drivers) for _ in range(len(df))]

Master Splinter doesn’t just run a taxi business. He is also a teacher. When he looks at the aggregated data by driver, he sees that Michelangelo is too busy partying to pick up fares, and when he does, he gets tipped very badly.

In [ ]:
earners = df.groupby("driver_id").aggregate(
    fare_count=pd.NamedAgg(column="pickup_dt", aggfunc="count"),
    distance=pd.NamedAgg(column="distance_km", aggfunc="sum"),
    fare_total=pd.NamedAgg(column="fare", aggfunc="sum"),
    tip_total=pd.NamedAgg(column="tip", aggfunc="sum"),
    avg_speed=pd.NamedAgg(column="speed_kph", aggfunc="mean")
)
earners["tip_rate"] = earners.apply(
    lambda x: x.tip_total/x.fare_total, axis=1
)
earners

Master Splinter decides he wants to implement an incentive program for his turtles. He's going to set a baseline tip rate for each driver and see how many of their rides exceed the baseline. Each driver has his own. 

He’s also worried that Donatello may be driving too carefully and Raphael too recklessly, so he’s going to set a baseline speed for each driver and make sure the driver stays within the range of that speed. 

Now, every time a driver exceeds his tip rate or keeps his speed to within 0.5 kph of his target, he gets an extra slice of pizza.

Let’s keep adding columns to the dataframe…

In [ ]:
drivers = {
    "Donatello": { "avg_speed": 18.5, "tip_rate": 0.16 },
    "Leonardo": { "avg_speed": 20.5, "tip_rate": 0.155 },
    "Michelangelo": { "avg_speed": 18.75, "tip_rate": 0.15 },
    "Raphael": { "avg_speed": 20.5, "tip_rate": 0.155 }
}

df["driver_target_speed"] = df.apply(lambda x: drivers[x.driver_id]["avg_speed"], axis=1)
df["driver_target_tip"] = df.apply(lambda x: drivers[x.driver_id]["tip_rate"], axis=1)
df["driver_within_target_speed"] = df.apply(lambda x: 
  (x.speed_kph > (x.driver_target_speed - 0.5) and x.speed_kph < (x.driver_target_speed + 0.5)),
  axis=1)
df["driver_exceeds_target_tip"] = df.apply(lambda x:
  x.tip/x.fare > x.driver_target_tip,
  axis=1)

pizza_slices = df.groupby("driver_id").aggregate(
    safe_speed=pd.NamedAgg(column="driver_within_target_speed", aggfunc="sum"),
    good_tip=pd.NamedAgg(column="driver_exceeds_target_tip", aggfunc="sum"),
)
pizza_slices["total_slices"] = pizza_slices.apply(
    lambda x: x.safe_speed + x.good_tip, axis=1
)
pizza_slices.sort_values("total_slices", ascending=False)

Just for reference, let’s look at the columns for this dataset.

In [ ]:
df.dtypes

Hmm. Of twelve columns in the dataset, five pertain to the _driver_, not to the actual _trip_ (though `driver_id` is arguably part of the trip… see below).

Let’s add one more wrinkle. Say Master Splinter wants to track customer data, too. He asks every customer to leave their name when they take a ride. Furthermore, he wants to send each customer a New Year’s card. Finally, he wants to implement a program where every fifth ride is free.

In [ ]:
Faker.seed(43)
def build_customer_data(n):
  profile = fake.simple_profile() 
  return {
      "id": str(n + 1).zfill(2),
      "name": profile["name"],
      "address": profile["address"].replace("\n", ", ")
  }
# For illustration purposes, let's assume there are only 10 customers.
customers = [ build_customer_data(n) for n in range(10) ]

np.random.seed(46)
def add_customer_columns(row):
  customer = np.random.choice(customers)
  row["customer_id"] = customer["id"]
  row["customer_name"] = customer["name"]
  row["customer_address"] = customer["address"]
  return row

df = df.apply(add_customer_columns, axis=1)

def is_ride_free(row):
  index = int(row.name)
  is_ride_free = False 
  if((len(df.iloc[0:index, :][df['customer_name'] == row["customer_name"]]) + 1) % 5 == 0):
    is_ride_free = True
  return is_ride_free

df["customer_rides_free"] = df.apply(is_ride_free, axis=1)

In [ ]:
df.dtypes

### PLEASE Can We Be Relational?

This is beginning to be grotesque. We have sixteen columns in Master Splinter’s dataset of taxi rides, but nine of them deal with the driver and customer. Granted, this is a contrived example, but the issue should be clear: the customer’s address should not belong in the trip data, and it’s probably the case that not even the driver’s target speed or tip should be in the trip data. 

The trip data should be just for the trips. The data about the drivers we can abstract into a different dataset, while keeping a link to the trips table so we know who drove a specific trip. I already effectively did this by creating the `drivers` dictionary above, which was a second dataset that I smushed into the trips data.

The same holds true for the customer data. The `customer` dictionary already has a lot of information in it, but then I smushed it into the trips dataset, too. We can abstract it back out but keep `customer_id` in the trips table to create a link.

Trips, drivers, and customers are related to each other, and we can even codify their relationships:

* a **trip** has a **driver** and a **customer** (or “belongs to”)
* a **driver** has many **trips**
* a **customer** has many **trips**

We can even formalize the data visually with an [entity relationship diagram](https://www.visual-paradigm.com/guide/data-modeling/what-is-entity-relationship-diagram/):

![An entity relationship diagram showing the three tables of the taxi data](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/master-splinter-taxi-erd.png)

Our data now tracks three **entities** in three **tables**. We can see that each entity has some properties, but that the `driver_id` and `customer_id` properties on the trip entity link drivers and customers.

We keep data connected to the entity to which it’s connected, and we can then run calculations on the fly to see if a customer rides free or if the driver gets a slice of pizza.

And now we have a relational database.

### OK, So What Is a Relational Database?

Well, basically the above. Data are separated into tables that are then linked together based on relationships. In this dataset, we have two “one-to-many” (or “has many”) relationships, but it is also possible to have “one-to-one” and “many-to-many” relationships. For example, if each driver had their own car that they always used, there could be a one-to-one relationship there, and properties pertaining to the car (model, year, color) would be stored in a cars table, as opposed to in the drivers table.

There are many ways to interact with relational databases, but perhaps the most common is by using the [Structured Query Language](https://en.wikipedia.org/wiki/SQL), or SQL. SQL reads somewhat easily for basic queries, like:

```sql
SELECT fare, tip FROM trips WHERE passengers > 3 ORDER BY pickup_dt;
```

This will give us the fare and tip amount of every trip with more than three passengers, sorted by pickup time. 

Additionally, we can [join tables](https://www.w3schools.com/sql/sql_join.asp) together to leverage the relationality with something like:

```sql
SELECT COUNT(*)
FROM trips
INNER JOIN drivers ON trips.driver_id=drivers.id
WHERE drivers.id='Leonardo'
AND trips.tip / trips.fare > drivers.target_tip;
```

This will count how many slices of pizza Leonardo gets for exceeding his target tip rate.

SQL is expressive and, again, somewhat easy to read, but different database servers implement slightly different syntax. This becomes clear below and paves the way for why, if we’re pythoning with a database, we probably want to get ourselves into using an [object-relational mapper](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping) as soon as we can, so we can get back to writing regular Python.

## Let’s Get Pokémon Data and Create a Database

Enough turtles. Let’s import an already existing database into our notebook. Brian Radomski and James Allen, building upon the [Veekun Pokédex](https://veekun.com/dex) put together a 14 table database of Pokémon for a course and then [shared it on GitHub](https://github.com/brianr852/Pokemon-Database). We’ve [forked it](https://github.com/columbia-data-club/Pokemon-Database) for safe-keeping. Radomski and Allen include a few reports in their repository and a MySQL dump of the database. 

In an ideal world, we would use the [`requests`](https://requests.readthedocs.io/en/latest/) library to download that dump (a database converted into a giant text file with a series of SQL commands that reproduces the database).  Since the dump would be in plain text, we would use the `text` property on the [`Response` object](https://requests.readthedocs.io/en/latest/api/#requests.Response) to read the data in. The code would be straightforward:

```py
import requests

pokemon_database_sql_dump_url = "https://raw.githubusercontent.com/columbia-data-club/Pokemon-Database/master/Dump20160519-1.sql"

r = requests.get(pokemon_database_sql_dump_url)
with open("pokemon_database_sql_dump.sql",'w') as f:
  f.write(r.text)
```

Unfortunately, we cannot use the dump as an actual database without processing it. SQLite can read an `.sql` file and convert it to a database, but it does not look like it’s possible to do that with the Frankenstein SQLite we have in Colab. 

Outside of Colab, then, I have already done the conversion. It involved using editing the dump (deleting every instance of `NO_AUTO_CREATE_USER,` which was [removed in MySQL 8.0.11](https://dev.mysql.com/doc/relnotes/mysql/8.0/en/news-8-0-11.html)) and then creating a database:

```sh
mysql.server start
mysql -p pokemon < Dump20160519-1.sql
```

Next, I installed and used the [`mysql-to-sqlite3`](https://pypi.org/project/mysql-to-sqlite3/) Python tool to convert the new MySQL database to a SQLite database:

```sh
pip install mysql-to-sqlite3
mysql2sqlite -u root -p -d pokemon -f pokemon.sqlite
```

This creates a binary `.sqlite` file. I added it to our [Meetings](https://github.com/columbia-data-club/meetings) repository, and we can download and use that, instead.

In [ ]:
import requests

pokemon_database_url = "https://github.com/columbia-data-club/meetings/raw/main/assets/data/pokemon.sqlite"

r = requests.get(pokemon_database_url) # create HTTP response object
with open("pokemon.sqlite","wb") as f:
  f.write(r.content)

Next, we can use `sqlite` to point to that database file and create an object that exists inside colab. This means we don’t need a database server and can keep everything local to our Colab sandbox. `SQLite` is available as the [`sql` magic extension](https://github.com/catherinedevlin/ipython-sql) in Colab.

In [ ]:
%load_ext sql
%sql sqlite:///pokemon.sqlite

Now we can run some basic SQL queries and make sure the database behaves like we hope it would. Paradoxically, we have a somewhat limited ability to access the metadata for the database. However, we can use the database entity relationship diagram provided by the Radomski and Allen to ground how we can reason about this Pokémon database.

![The entity relationship diagram for the Pokémon database](https://raw.githubusercontent.com/columbia-data-club/meetings/main/assets/images/pokemon-db-erd.png)

Let’s use the `PRAGMA` command to get information about the `pokemon` table.

In [ ]:
%sql PRAGMA table_info([pokemon]);

We can run some of the SQL commands the database’s authors provide to query the data, for example, to get a list of pokémon with attack, defense, and hp greater than 100. The resulting object, `strong_pokemon`, can also be converted to a pandas dataframe.

In [ ]:
%%sql strong_pokemon << SELECT pokemon.pok_id, pokemon.pok_name, 
  base_stats.b_atk, base_stats.b_def, base_stats.b_hp,
  base_stats.b_speed, base_stats.b_sp_atk, base_stats.b_sp_def
FROM pokemon
INNER JOIN base_stats
ON pokemon.pok_id = base_stats.pok_id
WHERE b_atk > 100 and b_def > 100 and b_hp > 100
GROUP BY pokemon.pok_id;

In [ ]:
# As an ipython-sql result
strong_pokemon

In [ ]:
# As a dataframe
strong_pokemon_df = strong_pokemon.DataFrame()
strong_pokemon_df

## Object-Relational Mappers

Thanks for suffering through that last section. Writing SQL queries (in native SQL like that) is no fun at all, and it takes us out of our programming rhythm. Luckily, [object-relational mappers (ORMs)](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping) exist. 

An ORM remaps a database’s structure into the class-based paradigm of object-oriented programming. Because an ERD tracks entities with properties, those entities can be reimagined as classes with properties. The relationships are mapped with some additional syntactic sugar, so instead of remembering SQL syntax and reasoning about the data in the database through whatever specific flavor of database you have, you can use more idiomatic programming structures.

There are several ORMs for Python, and some may even be already familiar to you. [SQLAlchemy](https://www.sqlalchemy.org/) seems to be the most popular, but there are smaller, more light-weight ones like [PeeWee](https://docs.peewee-orm.com/en/latest/). ORMs come especially in handy if you are building a web application where the code in your application relies on objects, but the database relies on entities and tables, and the popular web framework [Django](https://www.djangoproject.com/) has its [own ORM](https://docs.djangoproject.com/en/4.1/topics/db/queries/).

It's not a perfect comparison, but the crosswalk an ORM provides looks like this:

* A **Table** in the database becomes a **Model**, typically represented by a **Class**.
* A **Row** in the table becomes a **Model instance**, typically represented by a **Class instance**.
* A **Column** in the table becomes a **Attribute** of the model, typically represented by a **Class property**.

### Enter Pony

For the rest of this workshop, we’ll be using [Pony](https://ponyorm.org/) as our ORM. I do not have a lot of experience with Python ORMs, but I like how Pony embraces the more [functional programming](https://en.wikipedia.org//wiki/Functional_programming) side of Python in its use of [generators](https://docs.python.org/3/reference/expressions.html#generator-expressions) and [lambdas](https://docs.python.org/3/reference/expressions.html#lambda).

We installed Pony back in the first code cell of this notebook, but now let’s import it.

In [ ]:
from pony import orm


Once it’s imported, though, we need to do two things:

* Initialize a [`Database`](https://docs.ponyorm.org/api_reference.html#database-class) instance and make the connection (“bind it”) to the database (`pokemon.sqlite`).
* Create the mapping that turns the fourteen tables in the Pokémon database into a handful of classes we can work with in Python.

To build the mapping, we use [classes](https://www.w3schools.com/python/python_classes.asp) that inherit from Pony’s [`Entity`](https://docs.ponyorm.org/api_reference.html#entity-definition) object. We achieve two goals here: building the main interface for the ORM while also wiring things up correctly in the backend. Let’s start a starter mapping with just the `pokemon` table.

In [ ]:
# Initialize a Database instance and bind it to the `pokemon.sqlite` file.
starter_db = orm.Database()
starter_db.bind(provider='sqlite', filename="/content/pokemon.sqlite", create_db=False)

class Pokemon(starter_db.Entity): # Idiomatically, classes tend to be capitalized.
  id = orm.PrimaryKey(int, column="pok_id", auto=True) # INT(11)
  name = orm.Required(str, column="pok_name") # VARCHAR(79)
  height = orm.Optional(int, column="pok_height") # INT(11)
  weight = orm.Optional(int, column="pok_weight") # INT(11)
  base_experience = orm.Optional(int, column="pok_base_experience") # INT(11)

starter_db.generate_mapping(check_tables=True, create_tables=False)

The `generate_mapping` method connects the `Pokemon` class to the `pokemon` table in the database, which means now we can do some simple queries to show the syntax in action.

In [ ]:
# Let's get all the pokemon step by step to understand the API.
all_pokemon_query = orm.select(p for p in Pokemon)
type(all_pokemon_query)

Pony’s [`Query`](https://docs.ponyorm.org/api_reference.html#query-object) object contains the the database query, but to iterate over the results of the query, we have to add list-like syntax that turns the `Query` into a `QueryResult`, which is generally list-like.

In [ ]:
all_pokemon_list = all_pokemon_query[:]
print(type(all_pokemon_list))
print(type(all_pokemon_list[0]))
print(f"Name: {all_pokemon_list[0].name}, height: {all_pokemon_list[0].height}")
print(all_pokemon_list[0].to_dict())

The `.to_dict()` method on the `Entity` class converts the instance of the `Pokemon` class to a regular Python dictionary, meaning it’s pretty straightforward to now just feed everything into pandas with one line of code:

In [ ]:
pokemon_df = pd.DataFrame([pokemon.to_dict() for 
  pokemon in orm.select(p for p in Pokemon)[:]])
pokemon_df.describe()

Whether this is appropriate or not depends on your specific use case, but let’s start the db anew with a more thorough mapping of those fourteen tables, so we can leverage the relationality in the data.

In [ ]:
db = orm.Database()
db.bind(provider='sqlite', filename="/content/pokemon.sqlite", create_db=False)

class Pokemon(db.Entity):
  id = orm.PrimaryKey(int, column="pok_id", auto=True)
  name = orm.Required(str, column="pok_name")
  height = orm.Optional(int, column="pok_height")
  weight = orm.Optional(int, column="pok_weight")
  base_experience = orm.Optional(int, column="pok_base_experience")
  base_stats = orm.Optional("BaseStat")
  moves = orm.Set("Move", table="pokemon_moves", column="move_id")
  abilities = orm.Set("Ability", table="pokemon_abilities", column="abil_id")
  habitats = orm.Set("Habitat", column="hab_id", table="pokemon_evolution_matchup")
  types = orm.Set("Type", column="type_id", table="pokemon_types")

class BaseStat(db.Entity):
  _table_ = "base_stats"
  hp = orm.Optional(int, column="b_hp")
  attack = orm.Optional(int, column="b_atk")
  defense = orm.Optional(int, column="b_def")
  special_attack = orm.Optional(int, column="b_sp_atk")
  special_defense = orm.Optional(int, column="b_sp_def")
  speed = orm.Optional(int, column="b_speed")
  pokemon = orm.PrimaryKey(Pokemon, column="pok_id")  
  
class Ability(db.Entity):
  _table_ = "abilities"
  id = orm.PrimaryKey(int, column="abil_id")
  name = orm.Required(str, column="abil_name")
  pokemon = orm.Set(Pokemon, column="pok_id", table="pokemon_abilities")
 
class Move(db.Entity):
  _table_ = "moves"
  id = orm.PrimaryKey(int, column="move_id")
  name = orm.Required(str, column="move_name")
  power = orm.Optional(int, column="move_power") 
  pp = orm.Optional(int, column="move_pp") 
  accuracy = orm.Optional(int, column="move_accuracy")
  pokemon = orm.Set(Pokemon, column="pok_id", table="pokemon_moves")

class Habitat(db.Entity):
  _table_ = "pokemon_habitats"
  id = orm.PrimaryKey(int, column="hab_id")
  name = orm.Optional(str, column="hab_name")
  description = orm.Optional(str, column="hab_descript")
  pokemon = orm.Set(Pokemon, column="pok_id", table="pokemon_evolution_matchup")

class Type(db.Entity):
  _table_ = "types"
  id = orm.PrimaryKey(int, column="type_id")
  name = orm.Required(str, column="type_name")
  pokemon = orm.Set(Pokemon, column="pok_id", table="pokemon_types")

db.generate_mapping(check_tables=True, create_tables=False)

With the tables mapped to Python objects, we can reproduce the query from above, seeing a list of Pokémon with attack, defense, and hp greater than 100. Because the base stats are stored in another table (`base_stats`) and get loaded via a relationship to it, we can’t rely simply on the `.to_dict()` method anymore and have to be more explicit.

In [ ]:
def to_base_stat_dict(pokemon):
  return {
    "id": pokemon.id,
    "name": pokemon.name,
    "attack": pokemon.base_stats.attack,
    "defense": pokemon.base_stats.defense,
    "hp": pokemon.base_stats.hp,
    "speed": pokemon.base_stats.speed,
    "special_attack": pokemon.base_stats.special_attack,
    "special_defense": pokemon.base_stats.special_defense
  }

strong_pokemon = pd.DataFrame([ to_base_stat_dict(pokemon) for pokemon in 
                                Pokemon.select(lambda p: 
                                  p.base_stats.attack > 100 and 
                                  p.base_stats.defense > 100 and 
                                  p.base_stats.hp > 100)[:]
                              ])
strong_pokemon

Let’s implement a few other functions, etc., based on what Radomski and Allen provided in their [class report](https://github.com/columbia-data-club/Pokemon-Database/blob/master/Database%20Project.docx). For example, we can create a function that grabs a pokémon’s abilities and moves based on their name.

In [ ]:
def get_abilities(pokemon_name):
  pokemon = Pokemon.get(name=pokemon_name);
  return pokemon.abilities

def get_moves(pokemon_name):
  pokemon = Pokemon.get(name=pokemon_name);
  return pokemon.moves  

pokemon_name = "pikachu"
abilities = get_abilities(pokemon_name)
print(f"### Abilities for {pokemon_name}:")
for ability in abilities:
  print(ability.name)

pokemon_name = "venusaur"
moves = get_moves(pokemon_name)
print(f"### Moves for {pokemon_name}:")
for move in moves:
  print(move.name)

We can grab all the Pokémon who live in a certain habitat.

In [ ]:
cave = Habitat.get(name="cave")
for pokemon in cave.pokemon:
  print(pokemon.name)

I don’t know much about pokémon, but I find it curious that a pokémon can have more than one type, at least that’s how the database is designed:

```python
class Pokemon(db.Entity):
# ...
  types = orm.Set("Type", column="type_id", table="pokemon_types")

class Type(db.Entity):
# ...
  pokemon = orm.Set(Pokemon, column="pok_id", table="pokemon_types")
  ```

  Are there pokémon with more than one type?

In [ ]:
print(Pokemon.select(lambda p: orm.count(p.types) > 1).count())

#pokemon = Pokemon.select(lambda p: orm.count(p.types) > 1).random(1)[:][0]
#print(f"{pokemon.name.capitalize()} is a multi-type pokémon: {[type.name for type in pokemon.types]}")

And so on. We can continue playing these sorts of queries, etc., forever, but you can see now how to link up your notebook to a SQL database and, what’s more, how to leverage Pythonic generators and list comprehensions to query a database. 

You’ve also seen that flattening the relations in a database to a dataframe is possible, but not necessarily preferable. For example, adding the pokémon’s habitat to the dataframe of base stats is rather trivial, but how do we account for the fact that each pokémon has more than one ability? Or many moves? Or even many types? How do we account for that diversity in the two-dimensional space of a dataframe?

These questions have no real, correct answer; the solution depends on what the next steps are. For example, _internally_, the way the database keeps a many-to-many relationship straight is by using an intermediary table of ids from each table, thereby creating something that resembles what Hadley Wickham calls “[tidy data](http://vita.had.co.nz/papers/tidy-data.pdf),” where every row in the intermediate column is an observation on the nature of a relationship between a pokémon and a type. Hence, our dataframe could have a column, `type`, that then occasions the same pokémon appearing multiple times among the rows. Or each row could be a type, repeating as necessary to account for all the pokémon, who makeup a column. 

Structural questions like these, of course, are part of why relational databases exist in the first place! If each entity has its own properties attached to itself, and its relationships are clearly defined, then it should be the case that we can create any sort of dataframe that smushes those relationships into two dimensions, depending on what we need.